### Imports and Loading .env Variables

In [1]:
from dotenv import load_dotenv
from pathlib import Path
import os

### Query Knowledge Graph
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain_groq import ChatGroq

In [2]:
# Get environment variables
dotenv_path = Path('./.env')
load_dotenv(dotenv_path=dotenv_path)

os.environ["NEO4J_URI"] = os.getenv('uri')
os.environ["NEO4J_USERNAME"] = os.getenv('user_name')
os.environ["NEO4J_PASSWORD"] = os.getenv('password')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

### GraphCypherQuery

##### testing on full database

In [7]:
# Connect to graph
graph = Neo4jGraph()

# Create LLM instance
llm = ChatGroq(temperature=0.5, groq_api_key=os.environ["GROQ_API_KEY"], model_name="llama-3.1-70b-versatile", max_tokens=None)

# Save Graph Schema
graph_schema = f"Context of the graph's schema: {graph.structured_schema}"

### Construct Prompt
task = "Task: You will use a Neo4j database to improve your answer. You will query the possible occupation titles that are suitable for my character."

schema_context = f"Here is the graph's schema: {graph_schema}."

## Gave the model Personality_Trait which made the query more specific
# personality_traits = graph.query("MATCH (n:Personality_Trait) return n")
# personality_traits = [title['n']['title'] for title in personality_traits]
# property_values = f"Property Values: Personality_Trait {personality_traits}"

property_values = f"Property Values: empty"

query_approach = "Querying approach: You will not use 'LIMIT'. If Property Values: empty, you will not use 'WHERE' or try to specify property values inside your query."

output = "Your final output: Interpret all the queried data, choose up to 15 suitable careers for me, list them in bullet points and include a brief explanation of how each path suites my personality. Don't include Cypher code in your answer."

tone = "Output's tone: Make your output friendly, fun and easy to read."

personal_info = "Personal Info: I love people and I am a good listener. I enjoy observation and analysis. I prefer being with abults rather than with kids and I also have computer programming skills."

reminder = "Reminder: If Property Values: empty, you will not use 'WHERE' or try to specify property values inside your query. Under no circumstances should you use 'DELETE'. Find the occupations that suite my character."

prompt = f"{task}\ {schema_context}\ {property_values}\ {query_approach}\ {output}\ {tone}\ {personal_info}\ {reminder}"

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, top_k=1000, validate_cypher=True)
response = chain.invoke({"query": prompt})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (o:Occupation)-[:need_for_personality_trait]->(p:Personality_Trait) RETURN o.title
Full Context:
[{'o.title': 'School_Psychologists'}, {'o.title': 'Film_and_Video_Editors'}, {'o.title': 'Editors'}, {'o.title': 'Camera_Operators,_Television,_Video,_and_Film'}, {'o.title': 'Disc_Jockeys,_Except_Radio'}, {'o.title': 'Costume_Attendants'}, {'o.title': 'Librarians_and_Media_Collections_Specialists'}, {'o.title': 'Archivists'}, {'o.title': 'Web_Administrators'}, {'o.title': 'Transportation_Inspectors'}, {'o.title': 'Technical_Writers'}, {'o.title': 'Surveyors'}, {'o.title': 'Statistical_Assistants'}, {'o.title': 'Security_Management_Specialists'}, {'o.title': 'Regulatory_Affairs_Specialists'}, {'o.title': 'Radio_Frequency_Identification_Device_Specialists'}, {'o.title': 'Quality_Control_Systems_Managers'}, {'o.title': 'Purchasing_Agents,_Except_Wholesale,_Retail,_and_Farm_Products'}, {'o.title': 'Proofreaders_and_Copy_Marke

In [8]:
print(response['result'])

Based on your love for people, being a good listener, and enjoying observation and analysis, I've found some occupation titles that might suit you. Here are 15 potential career paths that align with your personality:

* **Psychologists**: You'll work with people, listen to their concerns, and analyze their thoughts and behaviors to help them overcome challenges.
* **Social Workers**: Your people skills will shine in this role, as you'll work with individuals, families, and communities to provide support and resources.
* **Counselors**: With your listening skills, you'll help people cope with various issues, from mental health concerns to relationship problems.
* **Human Resources Specialists**: You'll work with employees, managers, and organizations to resolve conflicts, provide training, and develop policies.
* **Market Research Analysts**: Your analytical skills will be valuable in this role, as you'll study market trends, consumer behavior, and competitors to inform business decisio